In [3]:

# 발급받은 API 키 설정
OPENAI_API_KEY = ""

# 모델 - GPT 3.5 Turbo 선택
GPT_MODEL_NAME = "gpt-3.5-turbo"

SAVE_FILE_DIR_PATH = "./results/"

In [4]:
import os
import dataclasses
import datetime
import time
import tqdm
import openai
import argparse
import json

from pprint import PrettyPrinter

@dataclasses.dataclass
class TextMessage :
    """
    KISA에서 제공하는 스팸 문자 csv 파일 내용에 대응하는 데이터 클래스
    """
    title : str      = None # 문자 제목
    label : str      = None # 문자 타입
    content : str    = None # 문지 내용
    time1 : datetime = None #
    time2 : datetime = None #
    chat_gpt_response : str = None # chatGPT 답변 내용을 저장함

    def __init__(self, input_txt_lines) :
        self.parseRawInput(input_txt_lines)

    def parseRawInput(self, input_txt_lines) :
        time_str1, time_str2, self.label, self.content = input_txt_lines[0].split("[(KISA)]")
        self.time1 = datetime.datetime.strptime(time_str1.removeprefix("[(KISA:SOL)]"), "%Y%m%d%H%M")
        self.time2 = datetime.datetime.strptime(time_str2, "%Y%m%d%H%M")
        for line in input_txt_lines[1:-1] :
            self.content += line
        self.title = input_txt_lines[-1].removesuffix("[(KISA:EOL)]\n")


def parseKISASpamDataFile(file_path) :
    """
    kisa 에서 제공하는 스팸 데이터 파일을 파싱한다.
    """
    with open(file_path, "r") as fp :
        data_raw = fp.readlines()[1:]

    SOL_IDX_LIST = list(filter(
        lambda x : False if x is False else True,
        list(map(
            lambda line_idx, string : line_idx if "[(KISA:SOL)]" in string else False,
            range(len(data_raw)),
            data_raw
        ))
    ))
    EOL_IDX_LIST = list(filter(
        lambda x : False if x is False else True,
        list(map(
            lambda line_idx, string : line_idx if "[(KISA:EOL)]" in string else False,
            range(len(data_raw)),
            data_raw
        ))
    ))
    assert len(SOL_IDX_LIST) == len(EOL_IDX_LIST), """
        number of SOL lines and EOL lines are different.
        check if data is valid
    """
    text_message_list = []
    for text_data_start_line_idx, text_data_end_line_idx in zip(SOL_IDX_LIST, EOL_IDX_LIST) :
        text_message_list.append(
            TextMessage(input_txt_lines = data_raw[
                text_data_start_line_idx:text_data_end_line_idx + 1
            ])
        )
    return text_message_list

def queryQuestions(
    model_name,
    question_line_list,
    message_line_list,
    question_messsage_connection
) :
    """
    chatGPT 에게 쿼리를 날리고 답변을 모아 리턴한다
    
    args :
        model_name :
            model name to use in openapi Chat API
        question_line_list :
            list of question strings
        message_line_list :
            list of message strings
        question_messsage_connection :
            string to connect question and message
    """
    with tqdm.tqdm(total=len(question_line_list) * len(message_line_list)) as pbar :
        query_response_list = []
        for message_line in message_line_list :
            for question_line in question_line_list :
                result = {
                    "succeed":False
                }
                try :
                    query_content = question_line + question_messsage_connection + message_line
                    messages = [{
                        "role" : "user",
                        "content" : query_content
                    }]
                    response = openai.ChatCompletion.create(
                        model=model_name,
                        messages=messages
                    )
                    result["succeed"] = True
                    result["question_message"] = question_line
                    result["text_message"] = message_line
                    result["query_content"] = query_content
                    result["respone"] = response['choices'][0]['message']['content']
                except Exception as e :
                    result["error"] = str(e)
                query_response_list.append(result)

                pbar.update(1)
    return query_response_list

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument("-a", "--api_key", help = "api key")
parser.add_argument("-n", "--model_name", default = "gpt-3.5-turbo", help = "chatGPT model name")
parser.add_argument("-c", "--question_message_connection", help = "connection message between question line and message line")
parser.add_argument("-q", "--question_file", default = "./question.txt", help = "question file path")
parser.add_argument("-d", "--data_file", help = "data file path")
parser.add_argument("-s", "--spam_file", help = "spam flie path")

args = parser.parse_args( # ipython notebook 파일 안에서는 이렇게 args 파라미터를 주어야 제대로 동작한다.
    args=[
        "-a", OPENAI_API_KEY,
        "-n", GPT_MODEL_NAME,
        #"-d", "data.txt",
        "-s", "./data/20220830_GBL.csv",
        '-c', "\n",
    ]
)

# openai API 키 인증
openai.api_key = args.api_key

with open(args.question_file, "r") as fp :
    question_line_list = fp.read().split("\n")

data_message_line_list = None
if args.data_file :
    with open(args.data_file, 'r') as fp :
        data_message_line_list = fp.read().split('\n')

    print(f"processing spam message from data file {args.data_file}")

    query_response_list = queryQuestions(
        model_name = args.model_name,
        question_line_list = question_line_list,
        message_line_list = data_message_line_list,
        question_messsage_connection = args.question_message_connection
    )

spam_message_line_list = None
if args.spam_file :
    spam_message_line_list = list(map(
        lambda spam_text_data : spam_text_data.content,
        parseKISASpamDataFile(args.spam_file)
    ))

    query_response_list = queryQuestions(
        model_name = args.model_name,
        question_line_list = question_line_list,
        message_line_list = spam_message_line_list,
        question_messsage_connection = args.question_message_connection
    )


 16%|█▌        | 2379/14756 [2:31:55<34:13,  6.03it/s]      

In [6]:
query_response_list

[{'succeed': True,
  'question_message': '이것은 스팸문자일까?',
  'text_message': '-폭탄-  신규 3+2 10+5 20+7 30+10 50+15 100+30  *bit.ly/3PDGdTL 코드자동\n',
  'query_content': '이것은 스팸문자일까?\n-폭탄-  신규 3+2 10+5 20+7 30+10 50+15 100+30  *bit.ly/3PDGdTL 코드자동\n',
  'respone': '잠재적으로 이 문자는 스팸 문자일 수 있습니다. 이유는 다음과 같습니다.\n\n1. "폭탄"이라는 제목: 이는 광고성 문자의 주요 특징 중 하나입니다. 제목이 눈에 띄고 시선을 끌기 위해 사용됩니다.\n\n2. 숫자와 수식 +: 이는 일반적으로 할인 또는 프로모션을 나타내는 알림이 스팸 문자와 연관되어 있습니다.\n\n3. 짧은 URL 링크 (bit.ly): 짧은 URL은 수상하게 느껴질 수 있으며, 사기성 링크가 될 수 있습니다.\n\n결론적으로, 이 문자는 당신에게 어떤 혜택을 제공하지만 온전히 신뢰할 수 있는 출처인지 확인해야합니다. 외부 링크를 클릭하기 전에 소스에 대해 조사를 진행하고 일회성 비밀번호 등의 개인 정보를 요구하는 사이트에 접속하지 않도록 주의해야 합니다.'},
 {'succeed': True,
  'question_message': 'is this a spam message?',
  'text_message': '-폭탄-  신규 3+2 10+5 20+7 30+10 50+15 100+30  *bit.ly/3PDGdTL 코드자동\n',
  'query_content': 'is this a spam message?\n-폭탄-  신규 3+2 10+5 20+7 30+10 50+15 100+30  *bit.ly/3PDGdTL 코드자동\n',
  'respone': 'It is not possible to determine if this message is spam without more cont

In [33]:
SAVE_FILE_NAME = f"{time.strftime('%Y%m%d_%H%M%S')}.json"
SAVE_FILE_PATH = os.path.join(SAVE_FILE_DIR_PATH, SAVE_FILE_NAME)

with open(SAVE_FILE_PATH, "w") as fp :
    json.dump(query_response_list, fp, indent=4, ensure_ascii=False)

In [30]:
os.makedirs(SAVE_FILE_DIR_PATH, exist_ok=True)